<div style="background-color:#1e1e1e; padding:25px; border-radius:10px; display:table; width:100%;">

  <!-- Left column: text -->
  <div style="display:table-cell; vertical-align:top; width:80%; padding-right:10px;">
    <h1 style="color:#4FC3F7; margin-bottom:10px;">Hands-On Part A: Loading a Pretrained Model</h1>

  <p style="font-size:18px; font-style:italic; color:#cccccc;">
    Workshop on T1D Simulator and Digital Twins for Personalized Care
  </p>

  <p style="font-size:15px; line-height:1.6; color:#dddddd;">
    This preparatory notebook demonstrates how to load a pretrained deep learning model 
    for type 1 diabetes (T1D) simulation, inspect its structure, and process patient data 
    using physiological approximations. We will use the data we process here for Part B.
  </p>
  </div>

  <!-- Right column: logo -->
  <div style="display:table-cell; vertical-align:middle; text-align:right; width:20%; padding-right:80px;">
    <img src="https://micelab.udg.edu/wp-content/uploads/2022/08/MICElab-letras_png-300x119.png" alt="MiceLab Logo" style="height:80px; border-radius:8px;">
  </div>

</div>


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
0. Imports &amp; Set-Up
</h2>

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../scripts"))

In [ ]:
%pip install tensorflow==2.13.0

In [ ]:
%pip install netron pandas matplotlib scikit-learn tqdm

In [ ]:
%pip install -U pip setuptools wheel
%pip uninstall -y typing_extensions
%pip install typing_extensions==4.12.2
%pip install -U ipykernel==6.29.5 jupyter-client traitlets

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import netron
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from tqdm import tqdm

import tensorflow as tf

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
1. Load Model
</h2>

Setup path and model name.

In [ ]:
path_to_model = "../misc/"
model_name = "pretrained-model.h5"
model_name = "vae_decoder_epoch_010.h5"

Load the .h5 using the `load_model` method from keras.

In [ ]:
model_filename = Path(path_to_model, model_name)
pretrained_vae_model = tf.keras.models.load_model(model_filename)

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
2. Inspect Model
</h2>

Let's explore the structure of the model we just loaded.

Previous to this workshop, we trained a VAE (Variational AutoEncoder) model with some cohort's data. A VAE consists of two Neural Networks, an Encoder and a Decoder. The encoder converts real examples of the target data we want to synthesize, into a compressed representation of itself. This compressed space is called a Latent Space. Then the decoder learns to do the opposite process, from the compressed representation, reconstruct the original real samples. 

Once this VAE model is trained, we extract the decoder and put it inside a GAN as its generator. The genereator actually has the same task as the decoder, converting a latent vector into a real samples. In the GAN training process we use the same Latent Space structure so that the pretrained model can correctly use the previously learned information.

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
2.1 Text summary
</h3>
The `.summary()` method is usually the first step when inspecting a model, it shows in plain text each Layer's
- Name and type
- Output Shape (first element is always `None` since it will correspond to the batch size)
- Number of Parameters
- Layers it connects to
- Whether it is set to trainable or  not (frozen, thus weights are not updated during backpropagation).

It ends by showing the total number of parameters of the entire model.

In [ ]:
pretrained_vae_model.summary(expand_nested=True, show_trainable=True)

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
2.2 Netron: Interactive Diagram
</h3>

Netron is a tool for visualizing neural network models interactively in your browser. The `netron.start(model_filename)` command launches a web-based viewer, allowing you to explore the model’s architecture, layers, and parameters directly in your browser.


After running the cell, open the provided link.

In [ ]:
netron.start(str(model_filename))

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
3. Load Data
</h2>

In [ ]:
path_df = Path("../misc/data_filtered.csv")

df = pd.read_csv(path_df)

display(df.head())

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
4. Physiological Approximations
</h2>


In this section, we use physiological models to approximate two key quantities for each patient:  
- **PI (Plasma Insulin):** The effective insulin in the bloodstream, calculated from both bolus and basal insulin delivery using a compartmental model based on the Hovorka equations[\[1\]](#ref1).  
- **RA (Rate of Appearance of Glucose):** The rate at which glucose enters the bloodstream after carbohydrate intake, also modeled using a compartmental approach[\[1\]](#ref1).

These approximations transform raw insulin and carbohydrate data into physiologically meaningful features, which are more suitable for downstream modeling and simulation. The resulting columns, `PI` and `RA`, are added to the patient dataframes and will be used as inputs for the deep learning model in the next steps.

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
4.1 Equations
</h3>

Here are the implementations for solving the differential equations in Python functions.

In [ ]:
def process_pi(df: pd.DataFrame, params_pi: dict) -> pd.DataFrame:
    """
    Process the 'PI' column of the dataframe using parameters from params_pi.

    Args:
        df (pd.DataFrame): Dataframe to process.
        params_pi (dict): Dictionary containing parameter values.

    Returns:
        pd.DataFrame: Dataframe with processed 'PI' column.
    """

    def system(y0, t, params, u_t_values):
        s1, s2, ifa = y0  # Corrected N806
        vi = params['VI']  # Corrected N806
        ke = params['Ke']  # Corrected N806
        tmaxi = params['tmaxI']  # Corrected N806
        kdia = 1 / tmaxi  # Corrected N806
        dt = params['dt']

        u_t = u_t_values[int(t / dt)] if int(t / dt) < len(u_t_values) else 0  # Corrected SIM108

        ds1 = u_t - kdia * s1  # Corrected N806
        ds2 = kdia * (s1 - s2)  # Corrected N806
        difa = (s2 / (tmaxi * vi)) - ke * ifa  # Corrected N806

        return [ds1, ds2, difa]

    # Fill missing data
    df['bolus'] = df['bolus'].fillna(0)

    bolus_by_id = {}
    # Group by ID and process each group
    for patient_id, group in tqdm(df.groupby('ID'), desc='Processing bolus PI', total=df['ID'].nunique()):  # Corrected N806
        u_t_values = group['bolus'].values / params_pi['dt']
        y0 = [0, 0, 0]  # Initial conditions
        t = np.arange(0, len(group) * params_pi['dt'], params_pi['dt'])  # Time vector

        # Solve ODE for the current group
        sol = odeint(system, y0, t, args=(params_pi, u_t_values), hmax=params_pi['dt'])
        bolus_by_id[patient_id] = sol[:, 2]  # Add solution for Ifa to the group's dataframe

    # Fill missing data
    df['basal'] = df['basal'].fillna(0)

    basal_by_id = {}
    # Group by ID and process each group
    for patient_id, group in tqdm(df.groupby('ID'), desc='Processing basal PI', total=df['ID'].nunique()):  # Corrected N806
        u_t_values = group['basal'].values / params_pi['dt']
        y0 = [0, 0, 0]  # Initial conditions
        t = np.arange(0, len(group) * params_pi['dt'], params_pi['dt'])  # Time vector

        # Solve ODE for the current group
        sol = odeint(system, y0, t, args=(params_pi, u_t_values), hmax=params_pi['dt'])
        basal_by_id[patient_id] = sol[:, 2]  # Add solution for Ifa to the group's dataframe

    for idx in df['ID'].unique():
        # Retrieve the index positions for the current sub_id
        index_positions = df[df['ID'] == idx].index

        # Retrieve the corresponding values from each dictionary
        boluses_values = bolus_by_id.get(idx, [0] * len(index_positions))
        basal_values = basal_by_id.get(idx, [0] * len(index_positions))

        pi_values = [-1*sum(x) for x in zip(boluses_values, basal_values, strict=False)]
        df.loc[index_positions, 'PI'] = pi_values

    return df


def process_ra(df: pd.DataFrame, params_ra: dict) -> pd.DataFrame:
    """
    Process the 'RA' column of the dataframe using parameters from params_ra.

    Args:
        df (pd.DataFrame): Dataframe to process.
        params_ra (dict): Dictionary containing parameter values.

    Returns:
        pd.DataFrame: Dataframe with processed 'RA' column.
    """
    dt = params_ra['dt']  # Extract time step for easier use later

    def system(y0, t, params, u_t_values):
        d1, d2 = y0  # Corrected N806
        tau_d = params['tau_D']  # Corrected N806
        a_g = params['A_G']  # Corrected N806

        u_t = u_t_values[int(t / dt)] if int(t / dt) < len(u_t_values) else 0  # Corrected SIM108

        # Rate equations
        dd1 = a_g * u_t - 1 / tau_d * d1  # Corrected N806
        dd2 = 1 / tau_d * d1 - 1 / tau_d * d2  # Corrected N806

        return [dd1, dd2]

    # Fill missing data
    df['carbs'] = df['carbs'].fillna(0)

    # Group by ID and process each group
    for _, group in tqdm(df.groupby('ID'), desc='Processing RA', total=df['ID'].nunique()):  # Corrected N806, B007
        u_t_values = group['carbs'].values
        y0 = [0, 0]  # Initial conditions for D1 and D2
        t = np.linspace(0, (len(group) - 1) * dt, len(group))  # Time vector

        # Solve ODE for the current group
        sol = odeint(system, y0, t, args=(params_ra, u_t_values), hmax=dt)

        # Compute Ra from D2
        d1, d2 = sol.T  # Corrected N806
        ra = d2 / params_ra['tau_D']  # Corrected N806

        # Store results in the group's dataframe
        group['RA'] = ra
        df.loc[group.index, 'RA'] = group['RA']

    return df

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
4.2 Parameters
</h3>

These two dictionaries contain the parameters for the PI and RA physiological models, extracted from [\[1\]](#ref1).
- **params_pi**:  
    Dictionary of parameters for the plasma insulin (PI) physiological model:  
    - `VI`: Insulin distribution volume (L/kg)  
    - `Ke`: Insulin elimination rate constant (1/min)  
    - `tmaxI`: Time-to-maximum insulin absorption (min)  
    - `dt`: Time step for simulation (min)  

- **params_ra**:  
    Dictionary of parameters for the rate of appearance (RA) of glucose model:  
    - `tau_D`: Time constant for glucose absorption (min)  
    - `A_G`: Fraction of carbohydrate absorbed  
    - `dt`: Time step for simulation (min)

In [ ]:
params_pi: dict[str, float] = {'VI': 0.12,
                               'Ke': 0.138,
                               'tmaxI': 55,
                               'dt': 5,
                               }
params_ra: dict[str, float] = {'tau_D': 40,
                               'A_G': 0.8,
                               'dt': 5
                               }

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
4.3 Let's solve
</h3>

In [ ]:
df = process_pi(df, params_pi)

df = process_ra(df, params_ra)

<h3 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
4.4 Plotting the Solved Equations
</h3>
Here we can see the original discrete signals (bolus and basal, and carbohydrate intake) plotted againsts the solved equations, which result in a continuus curve.

In [ ]:
length = 1000
length = min(length, len(df))
df_sliced = df.iloc[:length]
df_sliced = df.iloc[:length]

plt.figure(figsize=(6, 3))
plt.plot(
    df_sliced.index * 5,
    df_sliced['PI'],
    label='Solved PI',
    color='blue',
)
plt.plot(df_sliced.index * 5, df_sliced['bolus'], label='Bolus', color='red')
plt.plot(df_sliced.index * 5, df_sliced['basal'], label='Basal', color='green')

plt.xlabel('Time (minutes)')
plt.ylabel('Bolus Value')
plt.title(f'Solved PI vs. Bolus and Basal for Patient 11 (First {length} Points / {length*5} minutes)')
plt.legend(loc='best')
plt.show()

Don't worry about the negative insulin, we flip the sign intentionally.

In [ ]:
length = 1000
length = min(length, len(df))
df_sliced = df.iloc[:length]
df_sliced = df.iloc[:length]

plt.figure(figsize=(6, 3))
plt.plot(
    df_sliced.index * 5,
    df_sliced['RA']*10,
    label='Solved RA',
    color='blue',
)
plt.plot(df_sliced.index * 5, df_sliced['carbs'], label='Carbohydrates', color='red')

plt.xlabel('Time (minutes)')
plt.ylabel('Bolus Value')
plt.title(f'Solved RA (scaled x10) vs. Carbohydrate Intake for Patient 11 (First {length} Points / {length*5} minutes)')
plt.legend(loc='best')
plt.show()

For visualization purposes we have upscaled the solved RA vector.

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
5. Save Processed Data for Modeling
</h2>

In [ ]:
columns_to_save = ['ID', 'BG', 'PI', 'RA']
display(df[columns_to_save])
df[columns_to_save].to_csv('../misc/data_processed.csv', index=False)

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
References
</h2>
<a id="ref1"></a> [1] Hovorka R, Canonico V, Chassin LJ, Haueter U, Massi-Benedetti M, Orsini Federici M, et al. (2004). Nonlinear model predictive control of glucose concentration in subjects with type 1 diabetes. Physiological Measurement, 25(4), 905–920.